In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [2]:
!pip install kaggle

In [3]:
!mkdir ~/.kaggle

In [14]:
!cp kaggle.json ~/.kaggle/

In [15]:
!chmod 600 ~/.kaggle/kaggle.json

In [16]:
!kaggle datasets download -d blessondensil294/topic-modeling-for-research-articles

 61% 7.00M/11.4M [00:00<00:00, 69.2MB/s]
100% 11.4M/11.4M [00:00<00:00, 85.5MB/s]


In [17]:
!unzip /content/topic-modeling-for-research-articles.zip

Archive:  /content/topic-modeling-for-research-articles.zip
  inflating: test.csv                
  inflating: train.csv               


In [18]:
!python3 -m spacy download en_core_web_sm

2023-11-09 14:44:56.975628: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 14:44:56.975698: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 14:44:56.975741: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 14:44:58.273562: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
!pip install pyldavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 27.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn

In [19]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import PorterStemmer
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import re, string, random
import math
from nltk.corpus import stopwords
nltk.download('stopwords')

from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
spacy.load('en_core_web_sm')

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df = pd.read_csv('/content/train.csv')
df

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [21]:
stop = set(stopwords.words('english'))

In [22]:
def clean_document(doco):
    stop = set(stopwords.words('english'))

    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)
    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')
    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])
    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    doco_clean = [i for i in doco_clean if i not in stop]
    doco_clean = [i for i in doco_clean if i.isalpha()]
    doco_clean = [p.sub("", x).strip() for x in doco_clean]

    return doco_clean

In [23]:
review_clean = [clean_document(doc) for doc in df['ABSTRACT']]
sentences = [' '.join(r) for r in review_clean]
df['cleantext']=sentences
df.tail(20)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,cleantext
20952,20953,Spatial Variational Auto-Encoding via Matrix-V...,The key idea of variational auto-encoders (V...,1,0,0,1,0,0,key idea variational auto encoders vaes resemb...
20953,20954,Optimal Ramp Schemes and Related Combinatorial...,"In 1996, Jackson and Martin proved that a st...",1,0,0,0,0,0,jackson martin proved strong ideal ramp scheme...
20954,20955,Do Neural Nets Learn Statistical Laws behind N...,The performance of deep learning in natural ...,1,0,0,0,0,0,performance deep learning natural language pro...
20955,20956,Super-speeds with Zero-RAM: Next Generation La...,This article presents the novel breakthrough...,1,0,0,0,0,0,article presents novel breakthrough general pu...
20956,20957,Recoverable Energy of Dissipative Electromagne...,Ambiguities in the definition of stored ener...,0,1,0,0,0,0,ambiguities definition stored energy within di...
20957,20958,Elliptic Hall algebra on $\mathbb{F}_1$,We construct Hall algebra of elliptic curve ...,0,0,1,0,0,0,construct hall algebra elliptic curve mathbb u...
20958,20959,Approximate Bayesian inference with queueing n...,Queueing networks are systems of theoretical...,1,0,0,1,0,0,queueing networks systems theoretical interest...
20959,20960,Universal features of price formation in finan...,Using a large-scale Deep Learning approach a...,0,0,0,1,0,1,using large scale deep learning approach appli...
20960,20961,A New Tracking Algorithm for Multiple Colloida...,"In this paper, we propose a new algorithm ba...",0,1,0,0,0,0,paper propose new algorithm based radial symme...
20961,20962,Critical Percolation Without Fine Tuning on th...,We present numerical evidence that most two-...,0,1,0,0,0,0,present numerical evidence two dimensional sur...


In [24]:
data = df.cleantext.values.tolist()

In [25]:
data

['predictive models allow subject specific inference analyzing alterations neuroimaging data given subject data inference made two levels global identifiying condition presence local detecting condition effect extracted subject data global inference local inference used form subject specific effect maps rarely used existing models often yield noisy detections composed isolated islands article propose named rsm improve subject specific detections approaches particular binary classifiers rsm reduce noise due sampling error associated using examples train classifiers proposed method wrapper used different binary classifiers without information condition presence reconstruction maximum posteriori problem prior model whose parameters training data classifier specific fashion performed synthetically generated data data disease neuroimaging initiative adni database results data demonstrate using rsm yields higher detection using models directly bootstrap averaging analyses dataset show rsm al

In [26]:
data_words = [word_tokenize(sentence) for sentence in data]

In [27]:
data_words[:2]

[['predictive',
  'models',
  'allow',
  'subject',
  'specific',
  'inference',
  'analyzing',
  'alterations',
  'neuroimaging',
  'data',
  'given',
  'subject',
  'data',
  'inference',
  'made',
  'two',
  'levels',
  'global',
  'identifiying',
  'condition',
  'presence',
  'local',
  'detecting',
  'condition',
  'effect',
  'extracted',
  'subject',
  'data',
  'global',
  'inference',
  'local',
  'inference',
  'used',
  'form',
  'subject',
  'specific',
  'effect',
  'maps',
  'rarely',
  'used',
  'existing',
  'models',
  'often',
  'yield',
  'noisy',
  'detections',
  'composed',
  'isolated',
  'islands',
  'article',
  'propose',
  'named',
  'rsm',
  'improve',
  'subject',
  'specific',
  'detections',
  'approaches',
  'particular',
  'binary',
  'classifiers',
  'rsm',
  'reduce',
  'noise',
  'due',
  'sampling',
  'error',
  'associated',
  'using',
  'examples',
  'train',
  'classifiers',
  'proposed',
  'method',
  'wrapper',
  'used',
  'different',
  'bina

In [28]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['predictive', 'models', 'allow', 'subject', 'specific', 'inference', 'analyzing', 'alterations', 'neuroimaging', 'data', 'given', 'subject', 'data', 'inference', 'made', 'two', 'levels', 'global', 'identifiying', 'condition', 'presence', 'local', 'detecting', 'condition', 'effect', 'extracted', 'subject', 'data', 'global', 'inference', 'local', 'inference', 'used', 'form', 'subject', 'specific', 'effect', 'maps', 'rarely', 'used', 'existing', 'models', 'often', 'yield', 'noisy', 'detections', 'composed', 'isolated', 'islands', 'article', 'propose', 'named', 'rsm', 'improve', 'subject', 'specific', 'detections', 'approaches', 'particular', 'binary', 'classifiers', 'rsm', 'reduce', 'noise', 'due', 'sampling', 'error', 'associated', 'using', 'examples', 'train', 'classifiers', 'proposed', 'method', 'wrapper', 'used', 'different', 'binary', 'classifiers', 'without', 'information', 'condition', 'presence', 'reconstruction', 'maximum_posteriori', 'problem', 'prior', 'model', 'whose', 'param

In [29]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [30]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['predictive', 'model', 'allow', 'subject', 'specific', 'inference', 'analyze', 'alteration', 'neuroimage', 'datum', 'give', 'subject', 'data', 'inference', 'make', 'level', 'global', 'identifiye', 'condition', 'presence', 'local', 'detect', 'condition', 'effect', 'extract', 'subject', 'datum', 'global', 'inference', 'local', 'inference', 'use', 'form', 'subject', 'specific', 'effect', 'map', 'rarely', 'use', 'exist', 'model', 'often', 'yield', 'noisy', 'detection', 'compose', 'isolated', 'island', 'article', 'propose', 'name', 'improve', 'subject', 'specific', 'detection', 'approach', 'particular', 'binary', 'classifier', 'reduce', 'noise', 'due', 'sample', 'error', 'associate', 'use', 'example', 'train', 'classifier', 'propose', 'method', 'wrapper', 'use', 'different', 'binary', 'classifier', 'information', 'condition', 'presence', 'reconstruction', 'problem', 'prior', 'model', 'parameter', 'train', 'datum', 'classifi', 'specific', 'fashion', 'perform', 'synthetically', 'generate', 

In [31]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 3), (18, 1), (19, 3), (20, 1), (21, 1), (22, 2), (23, 1), (24, 2), (25, 6), (26, 1), (27, 1), (28, 5), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 4), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 4), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 2), (84, 5), (85, 1), (86, 1), (87, 5), (88, 1), (89, 1), (90, 2), (91, 7), (92, 1), (93, 2)]]


In [32]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('adni', 1),
  ('allow', 1),
  ('also', 1),
  ('alteration', 1),
  ('alzheimer_disease', 1),
  ('amyloid', 1),
  ('analysis', 1),
  ('analyze', 1),
  ('approach', 1),
  ('article', 1),
  ('associate', 1),
  ('average', 1),
  ('beta', 1),
  ('binary', 2),
  ('bootstrap', 1),
  ('cerebrospinal', 1),
  ('classifi', 1),
  ('classifier', 3),
  ('compose', 1),
  ('condition', 3),
  ('correlation', 1),
  ('cortical', 1),
  ('data', 2),
  ('database', 1),
  ('dataset', 2),
  ('datum', 6),
  ('demonstrate', 1),
  ('detect', 1),
  ('detection', 5),
  ('different', 1),
  ('directly', 1),
  ('disease', 1),
  ('due', 1),
  ('effect', 2),
  ('error', 1),
  ('examination', 1),
  ('example', 1),
  ('exist', 1),
  ('extract', 1),
  ('fashion', 1),
  ('fluid', 1),
  ('form', 1),
  ('generate', 1),
  ('give', 1),
  ('global', 2),
  ('high', 1),
  ('identifiye', 1),
  ('image', 1),
  ('improve', 2),
  ('improvement', 1),
  ('inference', 4),
  ('information', 1),
  ('initiative', 1),
  ('island', 1),
  (

In [33]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [34]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.073*"problem" + 0.045*"function" + 0.043*"set" + 0.041*"give" + '
  '0.039*"number" + 0.029*"solution" + 0.027*"prove" + 0.026*"class" + '
  '0.025*"case" + 0.023*"group"'),
 (1,
  '0.414*"system" + 0.099*"user" + 0.032*"resource" + 0.032*"share" + '
  '0.031*"decision" + 0.029*"run" + 0.021*"demand" + 0.021*"platform" + '
  '0.021*"validate" + 0.019*"assessment"'),
 (2,
  '0.195*"graph" + 0.125*"cluster" + 0.070*"spatial" + 0.060*"gas" + '
  '0.053*"galaxy" + 0.035*"filter" + 0.035*"feedback" + 0.028*"cloud" + '
  '0.023*"stellar" + 0.023*"arm"'),
 (3,
  '0.088*"matrix" + 0.075*"noise" + 0.071*"vector" + 0.050*"gradient" + '
  '0.048*"edge" + 0.046*"correlation" + 0.043*"classifier" + 0.043*"spectral" '
  '+ 0.040*"extract" + 0.034*"resolution"'),
 (4,
  '0.086*"robot" + 0.074*"particle" + 0.062*"able" + 0.048*"evaluation" + '
  '0.047*"significant" + 0.043*"initial" + 0.039*"sparse" + 0.037*"influence" '
  '+ 0.032*"integrate" + 0.032*"situation"'),
 (5,
  '0.103*"neural" +

In [35]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -13.394802104285285

Coherence Score:  0.32324201965617777


In [7]:
from gensim.corpora import MmCorpus
from gensim.models.ldamodel import LdaModel
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [40]:
save_path = "/content/drive/My Drive/"
lda_model.save(f"{save_path}/lda_model_research_topic.model")
MmCorpus.serialize(f"{save_path}/corpus_research_topic.mm", corpus)
id2word.save(f"{save_path}/id2word_research_topic.dict")

In [8]:
lda_model = LdaModel.load(f"lda_model_research_topic.model")
corpus = MmCorpus(f"corpus_research_topic.mm")
id2word = gensim.corpora.Dictionary.load(f"id2word_research_topic.dict")

In [9]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.294158  0.086864       1        1  27.879860
14     0.242676  0.144804       2        1  16.984738
0      0.178376  0.021063       3        1   9.330499
8      0.219837  0.079767       4        1   9.205650
16     0.117013 -0.050505       5        1   7.602691
12     0.098138  0.039906       6        1   5.138645
13     0.099540 -0.326369       7        1   4.691992
11     0.022796 -0.313148       8        1   3.273681
6     -0.033231 -0.011567       9        1   2.522681
9     -0.057364  0.013248      10        1   2.059793
1     -0.078429  0.019746      11        1   1.689999
3     -0.080386  0.019419      12        1   1.653601
4     -0.093618  0.024763      13        1   1.401230
19    -0.108264  0.030100      14        1   1.269233
2     -0.110354  0.027706      15        1   1.153877
5     -0.118305  0.033337      16        1   1.134031
17    -0.115436  0.032372      17        1   1.053487
10    -0.144257  0.040336      18        1   0.871164
18    -0.150741  0.041754      19        1   0.754488
15    -0.182151  0.046405      20        1   0.328659, topic_info=              Term          Freq         Total Category  logprob  loglift
174         system   9832.000000   9832.000000  Default   30.000  30.0000
107        network  10512.000000  10512.000000  Default   29.000  29.0000
63           model  20208.000000  20208.000000  Default   28.000  28.0000
83            show  13438.000000  13438.000000  Default   27.000  27.0000
75         problem  10085.000000  10085.000000  Default   26.000  26.0000
...            ...           ...           ...      ...      ...      ...
2656   composition      0.044612      1.274997  Topic20  -11.548   2.3652
2645     reference      0.044612      1.275003  Topic20  -11.548   2.3652
2392        metric      0.044613      1.275119  Topic20  -11.548   2.3651
3633      temporal      0.044612      1.275046  Topic20  -11.548   2.3652
2700  respectively      0.044612      1.275041  Topic20  -11.548   2.3652

[717 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1280     13  0.998611        able
1900     20  0.991039    absolute
179       1  0.999625    accuracy
685      10  0.996938    acoustic
1509     19  0.992250     acquire
...     ...       ...         ...
1308     13  0.997180        wide
1559     19  0.987463  widespread
332       1  0.887300        work
332       2  0.112491        work
1365      6  0.997792         yet

[733 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 15, 1, 9, 17, 13, 14, 12, 7, 10, 2, 4, 5, 20, 3, 6, 18, 11, 19, 16])